# Chapter 11: Training Deep Neural Networks

Melatih **Deep Neural Networks (DNN)** merupakan tantangan tersendiri dibandingkan dengan neural network dangkal. Semakin dalam sebuah jaringan, semakin besar pula kompleksitas optimisasi yang harus dihadapi selama proses training.

Deep Neural Networks mampu mempelajari representasi yang sangat kompleks, namun hal ini datang dengan sejumlah permasalahan serius yang dapat menghambat konvergensi, memperlambat pelatihan, atau bahkan menyebabkan model gagal belajar sama sekali.

Pada chapter ini, kita akan membahas berbagai **masalah utama dalam pelatihan DNN** serta **teknik-teknik modern** yang dikembangkan untuk mengatasinya, berdasarkan praktik terbaik dalam deep learning.


## Masalah Utama dalam Pelatihan Deep Neural Networks

Beberapa permasalahan utama yang sering muncul ketika melatih DNN antara lain:

1. **Vanishing dan Exploding Gradients**  
   Gradien menjadi terlalu kecil atau terlalu besar saat backpropagation.

2. **Fungsi Aktivasi yang Tidak Tepat**  
   Fungsi aktivasi tertentu dapat menyebabkan saturasi dan memperlambat training.

3. **Distribusi Input yang Tidak Stabil**  
   Perubahan distribusi aktivasi antar layer memperlambat konvergensi.

4. **Proses Training yang Lambat**  
   DNN membutuhkan optimisasi yang efisien agar dapat dilatih dalam waktu yang masuk akal.

5. **Overfitting**  
   Model dengan jutaan parameter sangat mudah menghafal data latih.

Chapter ini membahas solusi sistematis untuk setiap permasalahan tersebut.


## 1. Mengatasi Vanishing & Exploding Gradients

Masalah **vanishing gradient** terjadi ketika gradien menjadi sangat kecil saat disebarkan ke layer awal, sehingga bobot hampir tidak diperbarui.

Sebaliknya, **exploding gradient** terjadi ketika gradien menjadi sangat besar dan menyebabkan pembaruan bobot yang tidak stabil.

Kedua masalah ini sering muncul pada DNN karena:
- banyaknya layer,
- penggunaan fungsi aktivasi yang tidak tepat,
- inisialisasi bobot yang buruk.


In [ ]:
# Menggunakan Leaky ReLU
model = keras.models.Sequential([
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2)
])

# Menggunakan SELU untuk Self-Normalization
layer_selu = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")



### Inisialisasi Bobot yang Tepat (He Initialization)

Salah satu solusi paling efektif untuk mengatasi masalah gradien adalah menggunakan **inisialisasi bobot yang sesuai dengan fungsi aktivasi**.

**He Initialization** dirancang khusus untuk fungsi aktivasi ReLU dan variannya.

Prinsip dasarnya adalah menjaga varians aktivasi tetap stabil di seluruh layer selama forward dan backward pass.


In [ ]:
import tensorflow as tf
from tensorflow import keras

layer = keras.layers.Dense(
    10,
    activation="relu",
    kernel_initializer="he_normal"
)


Dengan He Initialization:
- gradien tidak cepat menghilang,
- training menjadi lebih stabil,
- konvergensi lebih cepat.

Teknik ini hampir menjadi standar de-facto untuk DNN berbasis ReLU.


## 2. Fungsi Aktivasi Non-Saturasi

Fungsi aktivasi seperti **sigmoid** dan **tanh** cenderung mengalami saturasi pada nilai input ekstrem, yang menyebabkan gradien mendekati nol.

Untuk mengatasi masalah ini, digunakan fungsi aktivasi **non-saturasi** yang tetap memiliki gradien signifikan pada sebagian besar domain input.


### Alternatif Fungsi Aktivasi Modern

Beberapa fungsi aktivasi non-saturasi yang populer meliputi:

- **Leaky ReLU**  
  Mengizinkan gradien kecil pada nilai negatif untuk mencegah neuron mati.

- **ELU (Exponential Linear Unit)**  
  Memberikan transisi halus di sekitar nol dan mempercepat konvergensi.

- **SELU (Scaled ELU)**  
  Dirancang untuk menciptakan *self-normalizing networks* jika dikombinasikan dengan inisialisasi yang tepat.


In [ ]:
# Leaky ReLU
model = keras.models.Sequential([
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2)
])

# SELU dengan self-normalization
layer_selu = keras.layers.Dense(
    10,
    activation="selu",
    kernel_initializer="lecun_normal"
)


Penggunaan fungsi aktivasi yang tepat:
- meningkatkan aliran gradien,
- mempercepat pelatihan,
- mengurangi risiko kegagalan konvergensi.


## 3. Batch Normalization

**Batch Normalization (BN)** adalah teknik yang menormalkan input pada setiap layer selama training.

BN mengurangi masalah yang disebut *internal covariate shift*, yaitu perubahan distribusi aktivasi antar layer selama training.


Manfaat utama Batch Normalization:
- mempercepat konvergensi,
- memungkinkan learning rate lebih besar,
- mengurangi sensitivitas terhadap inisialisasi bobot,
- bertindak sebagai regularisasi ringan.


In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])


Batch Normalization sangat dianjurkan untuk jaringan yang dalam dan kompleks.


## 4. Transfer Learning (Reusing Layers)

Transfer Learning memungkinkan kita memanfaatkan pengetahuan dari model yang sudah dilatih sebelumnya.

Alih-alih melatih model dari nol, kita:
- menggunakan layer awal dari model lama,
- hanya melatih layer akhir untuk tugas baru.


Pendekatan ini sangat efektif ketika:
- data berlabel terbatas,
- tugas baru masih mirip dengan tugas lama,
- training dari awal terlalu mahal secara komputasi.


In [ ]:
# Contoh konsep transfer learning

# model_A = keras.models.load_model("model_A.h5")
# model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
# model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# Membekukan layer lama
# for layer in model_B_on_A.layers[:-1]:
#     layer.trainable = False

# model_B_on_A.compile(
#     loss="binary_crossentropy",
#     optimizer="sgd",
#     metrics=["accuracy"]
# )


In [ ]:
# Misalkan kita memuat model A
# model_A = keras.models.load_model("my_model_A.h5")

# Membuat model B berdasarkan model A tanpa lapisan output terakhir
# model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
# model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# Membekukan (Freezing) lapisan yang digunakan kembali
# for layer in model_B_on_A.layers[:-1]:
#     layer.trainable = False

# Jangan lupa untuk compile setelah mengubah status trainable
# model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])


Transfer learning secara signifikan mengurangi waktu training dan meningkatkan performa awal model.


## 5. Optimizer Cepat

Optimizer menentukan bagaimana bobot diperbarui berdasarkan gradien.

SGD standar sering kali terlalu lambat untuk DNN besar.


Optimizer modern yang populer meliputi:
- **Momentum**
- **Nesterov Accelerated Gradient (NAG)**
- **RMSProp**
- **Adam**


In [ ]:
optimizer = keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999
)


Adam menggabungkan keunggulan Momentum dan RMSProp, sehingga sering menjadi pilihan default dalam deep learning.


## 6. Regularisasi untuk Mencegah Overfitting

DNN dengan kapasitas besar sangat rentan terhadap overfitting.

Salah satu teknik regularisasi paling efektif adalah **Dropout**.


Dropout bekerja dengan:
- menonaktifkan neuron secara acak selama training,
- memaksa jaringan untuk belajar representasi yang lebih robust,
- mengurangi ketergantungan pada neuron tertentu.


In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])


Dropout sering dikombinasikan dengan:
- Early Stopping,
- Batch Normalization,
- Data Augmentation.


## Kesimpulan (Chapter 11)

Berdasarkan praktik terbaik dalam pelatihan Deep Neural Networks, konfigurasi umum yang direkomendasikan adalah:

- Inisialisasi bobot: **He Initialization**
- Fungsi aktivasi: **ELU atau ReLU**
- Normalisasi: **Batch Normalization**
- Regularisasi: **Dropout + Early Stopping**
- Optimizer: **Adam atau Nadam**
- Learning rate: gunakan scheduler jika memungkinkan

Penerapan teknik-teknik ini secara konsisten dapat meningkatkan stabilitas, kecepatan konvergensi, dan performa akhir model deep learning.
